In [1]:
# https://www.kaggle.com/pavelvod/pytorch-starter-solution
!pip install torch

    100% |████████████████████████████████| 71.1MB 657kB/s ta 0:00:011


In [11]:
import utils

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
# giftCostsNp (5000, 100): [i,j] means the gift cost when i-th family choses j-th day.
row, col = 5000, 100
giftCostsNp = np.zeros((row, col))
familySizesNp = np.zeros((row, 1))

familyDataNp = utils.readDataDf().values

for i in range(row):
    familySizesNp[i] = familyDataNp[i, -1]
    for j in range(col):
        giftCostsNp[i,j] = utils.getCurGift(assignDay=j, 
                                          familyPreferencesList=familyDataNp[i, 1:-1].tolist(), 
                                          familySize=familySizesNp[i])

In [72]:
class Model(nn.Module):
    def __init__(self, giftCostsNp, familySizesNp):
        super().__init__()
        self.giftCostsTc = torch.from_numpy(giftCostsNp).type(torch.float32)
        self.familySizesTc = torch.from_numpy(familySizesNp).type(torch.float32)
        self.assignWeightsTc = torch.nn.Parameter(data=torch.Tensor(5000, 100), requires_grad=True)
        self.assignWeightsTc.data.uniform_(0,5) # weight initialization?
        
    def forward(self):
        assignProbTc = F.softmax(self.assignWeightsTc, dim=1)
#         print(self.giftCostsTc.shape, assignProbTc.shape)
        giftCostTotalTc = (self.giftCostsTc * assignProbTc).sum()
        
        NdTc = (assignProbTc.transpose(0,1) @ self.familySizesTc).sum(axis=1)
        NdTcCat = torch.zeros(101, dtype=torch.float32)
        NdTcCat[:-1] = NdTc
        
        cleanCostTotalTc = ((NdTc-125.)/400. * NdTc**(0.5+abs(NdTcCat[:-1]-NdTcCat[1:])/50.)).sum()
        return giftCostTotalTc, cleanCostTotalTc

In [83]:
def cost_function():
    return None

In [79]:
model = Model(giftCostsNp, familySizesNp)
best_score = 10e10
best_pos = None

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
for epoch in range(15000):
    optimizer.zero_grad()
    x, y = model()
#     print(x.shape, y.shape)
    loss = x + y
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(loss)
#         pos = model.assignWeightsTc.argmax(1).numpy()
#         a, b, v = cost_function(pos+1)
#         score = a + b
#         if score < best_score:
#             best_score = score
#             best_pos = pos
#         x = np.round(x.item(), 3)
#         y = np.round(y.item(), 3)
#         print(f"{epoch}\t{x}\t{y}\t{score}\t{a}\t{b}\t{v}")

tensor(34257688., grad_fn=<AddBackward0>)
tensor(12388296., grad_fn=<AddBackward0>)
tensor(11321018., grad_fn=<AddBackward0>)
tensor(10972253., grad_fn=<AddBackward0>)
tensor(10791172., grad_fn=<AddBackward0>)
tensor(10667394., grad_fn=<AddBackward0>)
tensor(10564663., grad_fn=<AddBackward0>)
tensor(10467239., grad_fn=<AddBackward0>)
tensor(10366871., grad_fn=<AddBackward0>)
tensor(10258669., grad_fn=<AddBackward0>)
tensor(10139164., grad_fn=<AddBackward0>)
tensor(10005852., grad_fn=<AddBackward0>)
tensor(9857358., grad_fn=<AddBackward0>)
tensor(9693415., grad_fn=<AddBackward0>)
tensor(9514839., grad_fn=<AddBackward0>)
tensor(9323325., grad_fn=<AddBackward0>)
tensor(9121209., grad_fn=<AddBackward0>)
tensor(8911199., grad_fn=<AddBackward0>)
tensor(8695971., grad_fn=<AddBackward0>)
tensor(8477827., grad_fn=<AddBackward0>)
tensor(8258630.5000, grad_fn=<AddBackward0>)
tensor(8039763., grad_fn=<AddBackward0>)
tensor(7822255., grad_fn=<AddBackward0>)
tensor(7606860.5000, grad_fn=<AddBackward